In [1]:
import os
import PIL
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import torch
from tqdm import tqdm
from PIL import Image
from datasets import Dataset, Image
from torch.utils.data import DataLoader
from typing import List, Union
from transformers import CLIPProcessor, CLIPModel


2025-03-25 19:48:05.927899: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 19:48:06.070079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-25 19:48:06.071701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-25 19:48:06.090928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 19:48:06.152435: I tensorflow/core/platform/cpu_feature_guar

In [2]:


device = "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

print(model.config)
for name, param in model.named_parameters():
    print(name, param.shape)

CLIPConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "openai/clip-vit-base-patch32",
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "bos_token_id": 0,
    "dropout": 0.0,
    "eos_token_id": 2,
    "model_type": "clip_text_model"
  },
  "transformers_version": "4.46.0",
  "vision_config": {
    "dropout": 0.0,
    "model_type": "clip_vision_model"
  }
}

logit_scale torch.Size([])
text_model.embeddings.token_embedding.weight torch.Size([49408, 512])
text_model.embeddings.position_embedding.weight torch.Size([77, 512])
text_model.encoder.layers.0.self_attn.k_proj.weight torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.k_proj.bias torch.Size([512])
text_model.encoder.layers.0.self_attn.v_proj.weight torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.v_proj.bias torch.Size([512])
text_model.encoder.layers.0.self_attn.q

In [ ]:

image_path = os.listdir("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Flicker_8k/Images/")
image_path = ['/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Flicker_8k/Images/' + path for path in image_path if '.jpg' in path]
image_path.sort()

# captions_df = pd.read_csv('captions.csv')
captions_df = pd.read_csv('/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Flicker_8k/captions.csv')



def encode_images(images: Union[List[str], List[PIL.Image.Image]], batch_size: int):
    def transform_fn(el):
        if isinstance(el['image'], PIL.Image.Image):
            imgs = el['image']
        else:
            imgs = [Image().decode_example(_) for _ in el['image']]
        return preprocess(images=imgs, return_tensors='pt')
        
    dataset = Dataset.from_dict({'image': images})
    dataset = dataset.cast_column('image',Image(decode=False)) if isinstance(images[0], str) else dataset       
    dataset.set_format('torch')
    dataset.set_transform(transform_fn)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    image_embeddings = []
    pbar = tqdm(total=len(images) // batch_size, position=0)
    with torch.no_grad():
        for batch in dataloader:
            batch = {k:v.to(device) for k,v in batch.items()}
            image_embeddings.extend(model.get_image_features(**batch).detach().cpu().numpy())
            pbar.update(1)
        pbar.close()
    return np.stack(image_embeddings)


def encode_text( text: List[str], batch_size: int):
    dataset = Dataset.from_dict({'text': text})
    dataset = dataset.map(lambda el: preprocess(text=el['text'], return_tensors="pt",
                                                        max_length=77, padding="max_length", truncation=True),
                            batched=True,
                            remove_columns=['text'])
    dataset.set_format('torch')
    dataloader = DataLoader(dataset, batch_size=batch_size)
    text_embeddings = []
    pbar = tqdm(total=len(text) // batch_size, position=0)
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            text_embeddings.extend(model.get_text_features(**batch).detach().cpu().numpy())
            pbar.update(1)
        pbar.close()
    return np.stack(text_embeddings)

vector_embedding = np.array(encode_images(image_path,32))



In [4]:

with open('/home/abhinav/Desktop/AIProjects/Internship/IIT Patna/CLIP/flicker8k_image_embeddings.pkl','wb') as f:
    pickle.dump(vector_embedding, f)